In [ ]:
"""
look for an isekai anime from 2022
look for an isekai anime with female harem
look for an anime like attack on titan
look for an anime like bleach and fullmetal alchemist
look for an sports anime with ecchi



request -> json
fix genre and tags
look for genre and tags

get genre and tags from like-animes
full search
if 0? 
"""

In [1]:
from openai import OpenAI
client = OpenAI()

def get_relevant_tags_and_genres(title: str, genres: list, tags: list) -> tuple[list, list]:
    response = client.responses.create(
        model="gpt-4.1",
        input="Return ONLY a python list of the 3 most relevant tags for the anime " + title + " with the genres " + str(genres) + " and the tags " + str(tags)
    )

    relevant_genres = [genres[0]]

    try:
        relevant_tags = eval(response.output_text)
    except:
        print("Error")
        print(response.output_text)
        relevant_tags = [tags[0]]
        
    return relevant_genres, relevant_tags

In [ ]:
from anilist import search_anime

search_results = search_anime(
    # search_term="Attack on titan",
    # genres=["Sports", "Ecchi"],  # Using single genre parameter
    tags=["Volleyball"],
    per_page=5
)

# # Print all results first
print("All results:")
for anime in search_results:
    print(f"- {anime['title']['english'] or anime['title']['romaji']}")
    print(f"  Genres: {', '.join(anime['genres'])}")
    print(f"  Tags: {', '.join([tag['name'] for tag in anime['tags']])}")
    print("-" * 60)

In [9]:
from typing import Optional, List, Literal
from pydantic import BaseModel
from crewai import Agent, Task, LLM
import os

# Initialize LLM (using GPT-4o from OpenAI)
llm = LLM(
    model="gpt-4.1",
    api_key=os.getenv('OPENAI_API_KEY'),
    max_tokens=2000,
    temperature=0
)

# 1 ── strict schema
class AnimeSearchParams(BaseModel):
    search_term: Optional[str] = None
    season:     Optional[Literal["WINTER", "SPRING", "SUMMER", "FALL"]] = None
    year:       Optional[int]  = None
    genres:     Optional[List[str]] = None
    tags:       Optional[List[str]] = None
    sort:       Optional[str] = None
    page:       Optional[int] = None
    per_page:   Optional[int] = None
    like_animes:   Optional[str] = None

    model_config = {"extra": "forbid"}

# 2 ── agent
mapper = Agent(
    name="AniListRequestMapper",
    role="Filter extractor",
    goal="Return only the filters in the user's request, as minimal JSON.",
    backstory="An anime librarian who knows the difference between genres and tags.",
    allow_delegation=False,
    llm=llm
)

# 3 ── task  – strict rules for genre vs tag + no nulls
OFFICIAL_GENRES = [
  "Action",
  "Adventure",
  "Comedy",
  "Drama",
  "Ecchi",
  "Fantasy",
  "Hentai",
  "Horror",
  "Mahou Shoujo",
  "Mecha",
  "Music",
  "Mystery",
  "Psychological",
  "Romance",
  "Sci-Fi",
  "Slice of Life",
  "Sports",
  "Supernatural",
  "Thriller"
]

map_request = Task(
    description=(
        "USER_REQUEST:\n"
        "{user_request}\n\n"
        "Produce **one JSON object** that validates against AnimeSearchParams.\n"
        "Rules:\n"
        f"• Only items in this list may appear in `genres`: {OFFICIAL_GENRES}.\n"
        "• Any other descriptive phrase (moods, sub‑genres like 'School Life', "
        "adjectives like 'Wholesome') must go into `tags`.\n"
        "• Title‑case every word (e.g. 'school‑life' → 'School Life').\n"
        "Don't necessarily include genres unless they are specified in the request.\n"
        "• Omit every key whose value would be null.\n\n"
        "Example:\n"
        "USER_REQUEST:  Recommend a dark fantasy from 2020.\n"
        "OUTPUT: {\"genres\": [\"Fantasy\"], \"tags\": [\"Dark\"], \"year\": 2020}"
    ),
    expected_output="A JSON dict with only the mentioned filters, no nulls.",
    output_json=AnimeSearchParams,
    agent=mapper,
)

In [10]:
from crewai import Crew, Process
from anilist import search_anime

# prompt = "an anime like ghost in the shell"
prompt = "I want an isekai anime with some comedy"


crew = Crew(
    agents=[mapper],
    tasks=[map_request],
    process=Process.sequential,
)
crew.kickoff(inputs={"user_request": prompt})

params = {k: v for k, v in map_request.output.json_dict.items() if v is not None}
print(params)

# from inspect import signature
# assert set(params).issubset(signature(search_anime).parameters)

{'genres': ['Comedy'], 'tags': ['Isekai']}


In [58]:
"""
AniList Search Tool for CrewAI
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
A robust, CrewAI‑compatible tool for querying the AniList public API.

Features
--------
* Search by keyword, season, year, genres, tags, and sort order.
* Optional "like_animes" boost: infer genres/tags from a seed list of
  anime titles to build a taste profile.
* Strict Pydantic **v2** schema for LLM‑friendly argument validation.
* Graceful HTTP error handling and sensible time‑outs.
* Zero side‑effect logging (debug statements removed).
"""

from __future__ import annotations

import logging
from typing import Any, Dict, List, Optional, Type, TypedDict

import requests
from pydantic import BaseModel, Field, field_validator
from crewai.tools import BaseTool

# --------------------------------------------------------------------------- #
#  Configuration & logging
# --------------------------------------------------------------------------- #

_ANILIST_API_URL: str = "https://graphql.anilist.co"
_REQUEST_TIMEOUT_S: int = 10

logger = logging.getLogger(__name__)

# --------------------------------------------------------------------------- #
#  Typed response helpers
# --------------------------------------------------------------------------- #

class Tag(TypedDict):
    id: int
    name: str
    rank: int
    isMediaSpoiler: bool

class Anime(TypedDict):
    id: int
    title: Dict[str, Optional[str]]
    genres: List[str]
    tags: List[Tag]
    averageScore: Optional[int]
    episodes: Optional[int]
    format: Optional[str]
    status: Optional[str]
    seasonYear: Optional[int]
    coverImage: Dict[str, str]

# --------------------------------------------------------------------------- #
#  Low‑level helper
# --------------------------------------------------------------------------- #

def _fetch_from_anilist(query: str, variables: Dict[str, Any]) -> Dict[str, Any]:
    """Send a GraphQL request to the AniList public API and return the JSON body."""
    try:
        resp = requests.post(
            _ANILIST_API_URL,
            json={"query": query, "variables": variables},
            timeout=_REQUEST_TIMEOUT_S,
        )
    except requests.exceptions.RequestException as exc:
        raise RuntimeError(f"Network error talking to AniList: {exc}") from exc

    if resp.status_code != 200:
        raise RuntimeError(
            f"AniList query failed (HTTP {resp.status_code}): {resp.text}"
        )

    payload = resp.json()
    if "errors" in payload:
        raise RuntimeError(f"AniList returned errors: {payload['errors']!r}")

    return payload

# --------------------------------------------------------------------------- #
#  Pydantic v2 input schema
# --------------------------------------------------------------------------- #

class SearchAnimeToolInput(BaseModel):
    """Arguments accepted by the :class:`SearchAnimeTool`."""

    search_term: Optional[str] = Field(
        None, description="Keyword to search in titles (Romaji or English).",
    )
    season: Optional[str] = Field(
        None,
        pattern=r"^(WINTER|SPRING|SUMMER|FALL)$",
        description='Season filter: "WINTER", "SPRING", "SUMMER", or "FALL".',
    )
    year: Optional[int] = Field(
        None,
        ge=1960,
        le=2100,
        description="Season year, e.g. 2024.",
    )
    genre: Optional[str] = Field(None, description="A single genre to filter by.")
    genres: Optional[List[str]] = Field(None, description="Multiple genres to filter by.")
    tags: Optional[List[str]] = Field(None, description="AniList tags to filter by.")
    sort: Optional[List[str]] = Field(
        default_factory=lambda: ["POPULARITY_DESC"],
        description='AniList sort keys, default `["POPULARITY_DESC"]`.',
    )
    page: int = Field(1, ge=1, description="1‑based page number.")
    per_page: int = Field(20, ge=1, le=50, description="Page size (max 50).")
    like_animes: Optional[str] = Field(
        None,
        description="Comma‑separated seed anime titles to build a taste profile.",
    )

    # Normalise blank strings to None so CrewAI can do `is not None`
    @field_validator("search_term", "season", "genre", "like_animes", mode="before")
    def _blank_to_none(cls, v):  # noqa: N805
        if isinstance(v, str) and not v.strip():
            return None
        return v

# --------------------------------------------------------------------------- #
#  CrewAI tool definition
# --------------------------------------------------------------------------- #

class SearchAnimeTool(BaseTool):
    """CrewAI tool **search_anime** – query AniList and return JSON anime list."""

    name: str = "search_anime"
    description: str = (
        "Search AniList for anime using free‑text title keywords, season/year, "
        "genres, tags, and sort order. Returns a JSON array of matching anime."
    )
    args_schema: Type[BaseModel] = SearchAnimeToolInput

    def _run(self, **kwargs) -> List[Anime]:  # noqa: N802
        params = SearchAnimeToolInput(**kwargs)

        variables: Dict[str, Any] = {
            "page": params.page,
            "perPage": params.per_page,
            "sort": params.sort,
        }

        if params.search_term:
            variables["search"] = params.search_term
        if params.season:
            variables["season"] = params.season
        if params.year:
            variables["seasonYear"] = params.year
        if params.genre:
            variables["genre"] = params.genre
        if params.genres:
            variables["genres"] = params.genres
        if params.tags:
            variables["tags"] = params.tags

        # Taste profile expansion
        if params.like_animes:
            taste_genres, taste_tags = self._build_taste_profile(params.like_animes)
            if taste_genres:
                variables["genres"] = sorted(
                    set(variables.get("genres", [])) | taste_genres
                )
            if taste_tags:
                variables["tags"] = sorted(
                    set(variables.get("tags", [])) | taste_tags
                )

        data = _fetch_from_anilist(_GRAPHQL_QUERY, variables)
        return data["data"]["Page"]["media"]

    async def _arun(self, **kwargs) -> Any:  # noqa: D401
        return self._run(**kwargs)

    # --------------------------------------------------------------------- #
    #  Private helpers
    # --------------------------------------------------------------------- #

    def _build_taste_profile(self, like_animes: str) -> tuple[set[str], set[str]]:
        """Aggregate genres & tags from a comma‑separated list of anime titles."""

        genres_acc: set[str] = set()
        tags_acc: set[str] = set()

        for raw in like_animes.split(","):
            title = raw.strip()
            if not title:
                continue
            hits = search_anime(title)
            if not hits:
                continue
            first = hits[0]
            g, t = get_relevant_tags_and_genres(
                first["title"].get("english") or first["title"].get("romaji"),
                first["genres"],
                [tag["name"] for tag in first["tags"]],
            )
            genres_acc.update(g)
            tags_acc.update(t)

        return genres_acc, tags_acc

# --------------------------------------------------------------------------- #
#  Static GraphQL query
# --------------------------------------------------------------------------- #

_GRAPHQL_QUERY: str = """
query (
  $search: String, $season: MediaSeason, $seasonYear: Int,
  $genre: String, $genres: [String], $tags: [String],
  $page: Int, $perPage: Int, $sort: [MediaSort]
) {
  Page(page: $page, perPage: $perPage) {
    media(
      search: $search, type: ANIME, season: $season,
      seasonYear: $seasonYear, genre: $genre, genre_in: $genres,
      tag_in: $tags, sort: $sort
    ) {
      id
      title { romaji english }
      genres
      tags { id name rank isMediaSpoiler }
      averageScore
      episodes
      format
      status
      seasonYear
      coverImage { medium }
    }
  }
}
""".strip()

# --------------------------------------------------------------------------- #
#  Example crew for quick manual testing
# --------------------------------------------------------------------------- #

from crewai import Agent, Crew, Task

anime_tool = SearchAnimeTool()

anime_researcher = Agent(
    role="Anime Researcher",
    goal="Find animes that match a user query.",
    backstory="You are a seasoned anime critic.",
    tools=[anime_tool],
)

recommendation_task = Task(
    description="Using AniList, compile a list of five anime that match the query.",
    expected_output="Five recommendations with one‑sentence justifications.",
    agent=anime_researcher,
)



In [61]:
crew = Crew(
    agents=[mapper, anime_researcher],
    tasks=[map_request, recommendation_task],
    process=Process.sequential,
)

In [66]:
result = crew.kickoff(inputs={"user_request": "I want an isekai anime with some comedy"})
print(result)

Here are five anime that fit the Comedy and Isekai genres:

1. **No Game No Life** - Follow the sibling duo Sora and Shiro as they conquer a world where every conflict is settled through high-stakes games, blending clever comedy with strategic gameplay. ![No Game No Life](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/b19815-sEOQ9yQaPKlk.jpg)

2. **KONOSUBA -God's blessing on this wonderful world!** - This hilarious take on the isekai genre features Kazuma and his dysfunctional party as they embark on comedic adventures in a fantasy realm filled with quirky characters and absurd situations. ![KONOSUBA](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx21202-qQoJeKz76vRT.png)

3. **Angel Beats!** - Set in the afterlife, this series expertly mixes action and comedy as a group of deceased students fight against a mysterious girl named Angel while uncovering the mysteries of their past lives. ![Angel Beats!](https://s4.anilist.co/file/anilistcdn/media/anime/cov

In [67]:
result = crew.kickoff(inputs={"user_request": "I want an anime like ghost in the shell"})
print(result)

1. **PSYCHO-PASS** - A psychological thriller set in a dystopian future, exploring themes of morality and social order through a police force that enforces law based on psychological analysis, similar to *Ghost In The Shell* in its exploration of technology and society. ![PSYCHO-PASS](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx13601-i42VFuHpqEOJ.jpg)

2. **Vivy: Fluorite Eye's Song** - A unique mix of action and music that follows an AI tasked with preserving the future of humanity, touching on the relationship between AI and humans, echoing *Ghost In The Shell's* themes of artificial intelligence. ![Vivy: Fluorite Eye's Song](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx128546-UIwyhuhjxmL0.jpg)

3. **Ghost In The Shell: Koukaku Kidoutai** - The original classic that set the tone for cyberpunk anime, delving into philosophical questions about identity and consciousness in a technologically advanced society. ![Ghost In The Shell](https://s4.anilis

In [63]:
result = crew.kickoff(inputs={"user_request": "I want an anime like akira"})
print(result)

Here are five anime recommendations based on your interest in "Akira":

1. **Texhnolyze**  
   A dark, thought-provoking series set in a dystopian city, exploring themes of human existence and evolution, much like "Akira" in its psychological depth and bleak outlook.

2. **Casshern Sins**  
   This visually stunning series combines action and deep philosophical questions in a post-apocalyptic world, paralleling "Akira's" exploration of humanity's struggles against dystopian transformations.

3. **Expelled From Paradise**  
   A unique blend of action and philosophy set in a dystopian future with advanced technology, reminiscent of "Akira" through its themes of artificial intelligence and societal collapse.

4. **Battle Angel Alita (GUNNM)**  
   A cyberpunk classic that delves into the life of a cyborg in a dystopian world, it shares "Akira's" thrilling action and intense emotional narratives about identity and resilience.

5. **Akira**  
   The quintessential cyberpunk film that defin

In [64]:
result = crew.kickoff(inputs={"user_request": "I want an anime like ghost int the shell"})
print(result)

1. **PSYCHO-PASS** - A thrilling exploration of a dystopian future where law enforcement uses a system to measure the likelihood of crime, paralleling the philosophical themes found in "Ghost in the Shell". 
   ![Cover Image](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx13601-i42VFuHpqEOJ.jpg)

2. **Vivy: Fluorite Eye’s Song** - This series intertwines action and music within a narrative about an AI's mission to protect humanity, echoing the existential questions raised in "Ghost in the Shell".
   ![Cover Image](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx128546-UIwyhuhjxmL0.jpg)

3. **Ghost in the Shell: Stand Alone Complex** - A series that delves further into the already established universe of "Ghost in the Shell", offering complex narratives that challenge the concept of humanity and technology.
   ![Cover Image](https://s4.anilist.co/file/anilistcdn/media/anime/cover/small/bx467-mBTtIoR13qs2.jpg)

4. **PLUTO** - A reimagining of the "Astro B